In [3]:
import numpy as np
from craco import uvfits_meta
import os

In [9]:
infile = "/CRACO/DATA_08/craco/SB056747/scans/35/20240105083536/b07.uvfits"
metafile = "/CRACO/DATA_00/craco/SB056747/SB56747.json.gz"
calfile = "/CRACO/DATA_00/craco/SB056747/cal/07/b07.aver.4pol.smooth.npy"
output_dir = "/CRACO/DATA_00/craco/gup037/test_uvfits_average/"

## Run the averaging script

In [21]:
tx1 = 10
outname1 = f"test.tx{tx1}.0_10.uvfits"
cmd1 = f"python ~/Codes/craco-python/src/craco/uvfits_average.py {infile} -metadata {metafile} -tstart 0 -tend 10 -tx {tx1} -outname {output_dir}/{outname1}"

tx2 = 3
outname2 = f"test.tx{tx2}.10_20.uvfits"
cmd2 = f"python ~/Codes/craco-python/src/craco/uvfits_average.py {infile} -metadata {metafile} -tstart 10 -tend 20 -tx {tx2} -outname {output_dir}/{outname2}"

In [22]:
os.system(cmd1)
os.system(cmd2)

Current position 1525920 writing 480
Wrote 480 to /CRACO/DATA_00/craco/gup037/test_uvfits_average//test.tx10.0_10.uvfits to make it from 1525920 1526400
Replaced UNKNOWN source with %s %s B1934-638_beam35 20h09m17.52909502s -66d30m24.19762081s
Current position 4560480 writing 1440
Wrote 1440 to /CRACO/DATA_00/craco/gup037/test_uvfits_average//test.tx3.10_20.uvfits to make it from 4560480 4561920
Replaced UNKNOWN source with %s %s B1934-638_beam35 20h09m17.52909502s -66d30m24.19762081s


0

In [23]:
os.system(f"ls -lrt {output_dir}")

total 29168
lrwxrwxrwx 1 gup037 nisuser       64 Feb 14 10:48 b07.uvfits -> /CRACO/DATA_08/craco/SB056747/scans/35/20240105083536/b07.uvfits
-rw-r--r-- 1 gup037 nisuser 15206400 Feb 14 10:57 b07.0_100.tx10.uvfits
-rw-r--r-- 1 gup037 nisuser  1549440 Feb 15 10:51 test.tx10.0_10.uvfits
-rw-r--r-- 1 gup037 nisuser  4584960 Feb 15 10:51 test.tx3.10_20.uvfits


0

## Now read the two files

In [28]:
fo = uvfits_meta.open(infile, metadata_file = metafile)
fa1 = uvfits_meta.open(output_dir + "/" + outname1)
fa2 = uvfits_meta.open(output_dir + "/" + outname2)

In [29]:
d_o1, uvws_o1 = next(fo.fast_time_blocks(nt = 10, fetch_uvws=True, keep_all_baselines=True))
d_o2, uvws_o2 = next(fo.fast_time_blocks(nt = 3, istart=10, fetch_uvws=True, keep_all_baselines=True))

In [40]:
d_o1.shape, d_o2.shape, len(uvws_o1), len(uvws_o1[0]), len(uvws_o1[0][list(uvws_o1[0].keys())[0]])

((435, 1, 1, 1, 288, 1, 10), (435, 1, 1, 1, 288, 1, 3), 10, 435, 3)

In [32]:
d_a1, uvws_a1 = next(fa1.fast_time_blocks(nt = 1, fetch_uvws=True, keep_all_baselines=True))
d_a2, uvws_a2 = next(fa2.fast_time_blocks(nt = 1, fetch_uvws=True, keep_all_baselines=True))

In [33]:
d_a1.shape, d_a2.shape

((435, 1, 1, 1, 288, 1, 1), (435, 1, 1, 1, 288, 1, 1))

## Now average the data and uvw values yourselves as you think they should have been

In [65]:
def average_data_and_uvws(data, uvws, tx):
    elements_to_check = ['UU', 'VV', 'WW']
    avg_block = data[..., 0].filled(fill_value = 0)
    nsum = np.zeros(avg_block.shape, dtype='uint32') + ~data[..., 0].mask
    avg_uvws = {}
    for blid in uvws[0].keys():
        avg_uvws[blid] = []
        avg_uvws[blid].append(uvws[0][blid]['UU'])
        avg_uvws[blid].append(uvws[0][blid]['VV'])
        avg_uvws[blid].append(uvws[0][blid]['WW'])
        avg_uvws[blid] = np.array(avg_uvws[blid])
    navg = 1
    for it in range(1, tx):
        avg_block += data[..., it].filled(fill_value = 0)
        nsum += ~data[..., it].mask

        for blid in uvws[it].keys():
            avg_uvws[blid][0] += uvws[it][blid]['UU']
            avg_uvws[blid][1] += uvws[it][blid]['VV']
            avg_uvws[blid][2] += uvws[it][blid]['WW']
        navg += 1 
    
    valid = nsum > 0
    avg_block[~valid] = 0
    avg_block[valid] /= nsum[valid]

    for blid in avg_uvws.keys():
        avg_uvws[blid] /= navg

    return avg_block, avg_uvws


In [115]:
avg_block1, avg_uvws1 = average_data_and_uvws(d_o1, uvws_o1, 10)
avg_block2, avg_uvws2 = average_data_and_uvws(d_o2, uvws_o2, 3)

## Check if the averged data are the same as in the averaged file

In [72]:
np.all(avg_block1 == d_a1[..., 0])

True

In [116]:
np.all(avg_block2 == d_a2[..., 0])

True

## Check if the averaged uvws are the same as in the averaged file

In [117]:
def compare_uvws(uvws_a1, avg_uvws1):
    all_true = True
    for ii, blid in enumerate(list(avg_uvws1.keys())):
        file_blid = list(uvws_a1[0].keys())[ii]
        assert file_blid == blid
        a = avg_uvws1[blid][0] == uvws_a1[0][blid]['UU']
        b = avg_uvws1[blid][1] == uvws_a1[0][blid]['VV']
        c = avg_uvws1[blid][2] == uvws_a1[0][blid]['WW']
        all_true = all_true and (a==b) and (b==c)
    return all_true


In [118]:
compare_uvws(uvws_a1, avg_uvws1)

True

In [119]:
compare_uvws(uvws_a2, avg_uvws2)

True

## Check if the INTTIM is sensible

In [100]:
vis_o1 = next(fo.fast_raw_blocks(nt=1))
vis_o2 = next(fo.fast_raw_blocks(nt=1,istart = 10))
vis_a1 = next(fa1.fast_raw_blocks(nt=1))
vis_a2 = next(fa2.fast_raw_blocks(nt=1))

In [113]:
np.isclose(vis_o1['INTTIM'][0][0] * tx1, vis_a1['INTTIM'][0][0])

True

In [114]:
np.isclose(vis_o2['INTTIM'][0][0] * tx2, vis_a2['INTTIM'][0][0])

True

## Check if the DATE is sensible

In [158]:
vis_o1 = next(fo.fast_raw_blocks(nt=tx1, raw_date=False))
vis_o2 = next(fo.fast_raw_blocks(nt=tx2,istart = 10))
vis_a1 = next(fa1.fast_raw_blocks(nt=1, raw_date = False))
vis_a2 = next(fa2.fast_raw_blocks(nt=1))

In [159]:
np.allclose(vis_o1['DATE'].mean(axis=0)[0] * 1, vis_a1['DATE'][0][0] * 1)

True

In [160]:
np.allclose(vis_o2['DATE'].mean(axis=0)[0] * 1, vis_a2['DATE'][0][0] * 1)

True

In [150]:
fo.header['PZERO4'] == fa1.header['PZERO4'] and fa1.header['PZERO4'] ==  fa2.header['PZERO4']

True

## Check if other unchanged parameters are the same

In [166]:
params = ['BASELINE', 'FREQSEL', 'SOURCE']
for param in params:
    print(param, vis_o1[param][0][0] == vis_a1[param][0][0], vis_o2[param][0][0] == vis_a2[param][0][0])

BASELINE True True
FREQSEL True True
SOURCE True True
